In [80]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.


In [2]:
%sendtofile --source ../basicsockets/minimalmdns.py

Sent 82 lines (2967 bytes) to minimalmdns.py.


In [ ]:
# This is the code that runs the neopixels
# mosquitto_pub -h mqtt.local -t esp32j/3 -m 0a1a2a
# (see below for the code that goes on the drums with the accelerometer)

In [81]:
%sendtofile main.py

import machine, time, network, neopixel

wifinames = ["DoESLiverpool-MQTT", "DoESLiverpool"]
wifipassword = "decafbad00"
mqttbrokername = "mqtt.local"

from machine import Pin, ADC
import time

neos = [ ]
neos.append(neopixel.NeoPixel(machine.Pin(12), 74))
neos.append(neopixel.NeoPixel(machine.Pin(13), 74))
pled = machine.Pin(2, machine.Pin.OUT)

def fillallneos(c):
    for neo in neos:
        neo.fill(c)
        neo.write()
    pled.value(1-pled.value())
fillallneos((10,0,0))


time.sleep(1.5)  # wait to turn on wifi
si = network.WLAN(network.STA_IF)
si.active(True)
sw = [(-s[3], s[0].decode())  for s in si.scan()]
sw.sort()
print(sw)
for s, wifiname in sw:
    if wifiname in wifinames:
        break
else:
    wifiname = wifinames[0]
del sw
    

print("Connecting to", wifiname)
si.connect(wifiname, wifipassword)
while not si.isconnected():
    time.sleep(0.1)
fillallneos((0,10,0))
print("connected to wifi")


if mqttbrokername[-6:] == ".local":
    from minimalmdns import mdnshostnametoipnumber
    mqttbroker = mdnshostnametoipnumber(si, mqttbrokername)
    fillallneos((0,0,10))
    print("mDNS completed", mqttbrokername, mqttbroker)
else:
    mqttbroker = mqttbrokername


from umqtt.robust import MQTTClient
client = MQTTClient("neopixelesp44", mqttbroker, 1883)
for i in range(100):
    try:
        client.connect()
        break
    except OSError as e:
        print(e)
fillallneos((0,10,0))
print("connected to mqtt")


def scallback(topic, msg):
    print((topic, msg))
    try:
        r,g,b = int(msg[:2], 16), int(msg[2:4], 16), int(msg[4:6], 16)
    except ValueError:
        print("ignore")
        return
    try:
        neo = neos[int(topic[-1:])]
    except Exception as e:
        print(e)
        neo = None
    if neo is not None:
        print("filling", neo)
        neo.fill((r,g,b))
        neo.write()
    else:
        print("fill all")
        fillallneos((r,g,b))

client.set_callback(scallback)
client.subscribe(b"esp32j/#")

while 1:
    client.check_msg()
    time.sleep_ms(5)

Sent 91 lines (2029 bytes) to main.py.


In [72]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.


In [ ]:
# This is the code that gets saved onto the drum accelerometer
# mosquitto_sub -v -h mqtt.local -t "esp32black/maxaccel"

In [59]:
%sendtofile --source ../basicsockets/minimalmdns.py

Sent 82 lines (2967 bytes) to minimalmdns.py.


In [79]:
%sendtofile main.py

import machine, time, network

wifinames = ["DoESLiverpool-MQTT", "DoESLiverpool"]
wifipassword = "decafbad00"
mqttbrokername = "mqtt.local"

from machine import Pin, I2C
import time, ustruct

time.sleep(0.1)

i2c = I2C(scl=Pin(15), sda=Pin(4), freq=450000)
#    if 0x6B in ads and 0x1E in ads:
#        res.append("SDOF GyAccMag")
pled = machine.Pin(2, machine.Pin.OUT)

        
# turn on accelerometer reg(0x20)=(ODR_XL,FS_XL,BW_SCAL_ODR,BW_XL)=110 00 0 00 should give 952Hz, but is overridden by gyros ODR 
# there are various further settings of FIFO and High and Low pass filters
i2c.writeto(0x6B, b'\x20\xC0')  
i2c.writeto(0x1E, b'\x22\x00')  

def readaccelerometer(timeout=1000):
    cs, stm, ds = 0x6B, 1, b"\x28" 
    t0 = time.ticks_ms()
    while True:   # loop to wait for readings to be ready (at 60Hz)
        i2c.writeto(cs, b'\x27')
        st = ord(i2c.readfrom(cs, 1))  # (IG_XL,IG_G,INACT,BOOT_STATUS,TDA,GDA,XLDA) states whether a reading is ready
        if st & stm:
            break
        if time.ticks_ms() - t0 > timeout:
            return (0, 0, 0)
    i2c.writeto(cs, ds)
    s = i2c.readfrom(cs, 6)
    sv = ustruct.unpack("<hhh", s)
    return sv


pled.value(1)
time.sleep(1.5)  # wait to turn on wifi
si = network.WLAN(network.STA_IF)
si.active(True)
sw = [(-s[3], s[0].decode())  for s in si.scan()]
sw.sort()
print(sw)
for s, wifiname in sw:
    if wifiname in wifinames:
        break
else:
    wifiname = wifinames[0]
del sw
pled.value(1) 


print("Connecting to", wifiname)
si.connect(wifiname, wifipassword)
while not si.isconnected():
    time.sleep(0.1)
pled.value(0) 
print("connected to wifi")


if mqttbrokername[-6:] == ".local":
    from minimalmdns import mdnshostnametoipnumber
    mqttbroker = mdnshostnametoipnumber(si, mqttbrokername)
    print("mDNS completed", mqttbrokername, mqttbroker)
else:
    mqttbroker = mqttbrokername
pled.value(1) 


from umqtt.robust import MQTTClient
client = MQTTClient("esp32black99", mqttbroker, 1883)
for i in range(100):
    try:
        client.connect()
        break
    except OSError as e:
        print(e)
pled.value(0) 
print("connected to mqtt")


import math
tdiff = 100
t0 = time.ticks_ms()
xmax = None
while True:
    #client.check_msg()
    k = readaccelerometer()
    x = math.sqrt((k[0]/16384)**2 + (k[1]/16384)**2 + (k[2]/16384)**2)
    if xmax is None or x > xmax:
        xmax = x
    t1 = time.ticks_ms()
    if t1 > t0 + tdiff and xmax is not None:
        print(xmax)
        client.publish(b"esp32black/maxaccel", str(xmax))
        xmax = None
        t0 = t1


Sent 99 lines (2573 bytes) to main.py.
